<a href="https://colab.research.google.com/github/mylethidiem/zero-to-hero/blob/main/preAIO25/external/VisionTransformer_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vision Transformer**

In [2]:
# Install packages
!pip install torchvision==0.17.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
#
import torch
import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import DataLoader
import torchvision
import torchvision.transform as transform

ModuleNotFoundError: No module named 'torchvision.transform'

In [ ]:
#?
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## LOAD DATA CIFAR10

What is [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)?




In [ ]:
batch_size = 4

# Why need transform??
transform =  transform.Compose(
    [
        transform.Resize((224, 224)),
        transform.ToTensor(),
        transform.Normalize([(0.5, 0.5, 0.5), (0.5, 0.5, 0.5)]) #?
    ]
)

#download
test_dataset = torch.vision.dataset.CIFAR10(root='./data', train=False, transform=transform, download=True)

# split data train into val and train
train_dataset = torchvision.dataset.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_subset_size = int(len(train_traindataset) * 0.9)
val_subset_size = len(train_traindataset) - train_subset_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_subset_size,val_subset_size ])


# create dataloader contain batches of data to train(mini-batch)
train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=True, nums_worker=2)#?
val_loader = DataLoader(dataset=val_subset, batch_size=batch_size, shuffle=False, nums_worker=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, nums_worker=2)


## Build Model

In [ ]:
# Patch Embedding
# usage, parameters?
class PatchEmbedding(nn.Module):
  def __init__(self, embed_dim=512, patch_size=16, image_size=224):
    super().__init__()
    # 1 layer conv2d
    # image have 3 channels
    # why we don't need a bias?
    # why embed dim have that value?
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=embed_dim, kernel_size=patch_size, stride=patch_size, bias=False)

  def forward(self, x):
    x = self.conv1()

    # why we need reshape?
    print(f"x shape before reshape: {x.shape}") # shape [*, width, grid, grid]
    x = x.reshape(x.shap[0], x.shape[1], -1)
    print(f"x shape after reshape: {x.shape}") # shape [*, width, grid ** 2]

    # why need permute x shape?
    x.permute(0, 2, 1)
    return x


In [ ]:
# example
patch_embedding = PatchEmbedding() # model
x = torch.randn(1, 3, 224, 224)

x = patch_embedding(x)
print(x.shape)

In [ ]:
# Patch Positional Embedding
# usage, parameters?
class PatchPositionalEmbedding(nn.Module):
  def __init__(self, embed_dim=512, patch_size=16, image_size=224):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=embed_dim, kernel_size=patch_size, stride=patch_size, bias=False)
    scale = embed_dim ** -0.5 #?
    print(embed_dim)
    data = scale * torch.randn((image_size // patch_size) ** 2, embed_dim)#?
    self.positional_embedding = nn.Parameter(data)

  def forward(self, x):
    x = self.conv1(x) # shape = [*, width, grid]
    x = x.reshape(x.shape[0], x.shape[1], -1) # shape = [*, width, grid ** 2]
    x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]

    x = x + self.positional_embedding.to('cpu') #?
    return x


In [ ]:
patchpos_embedding = PatchPositionalEmbedding()
x = torch.randn(1, 3, 224, 224)

out = patchpos_embedding(x)
print(out.shape)

In [ ]:
# Adding [CLS] token
class PatchPostionalEmbeddingCLSToken(nn.Module):
  def __init__(self, embed_dim=512, patch_size=16, image_size=224):
    super().__init__(self)
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=embed_dim, kernel_size=patch_size, stride=patch_size, bias=False)

    scale = embed_dim ** -0.5

    data_class_embedding = scale * torch.randn(embed_dim)
    self.class_embedding = nn.Parameter(data_class_embedding)

    #?
    data_positional_embedding = scale * torch.randn((image_size // patch_size) ** 2 + 1, embed_dim)
    self.positional_embedding = nn.Parameter(data_positional_embedding)

  def forward(self, x):
    x = self.conv1(x) # shape = [*, width, grid, grid]
    x = x.reshape(x.shape[0], x.shape[1], -1) # shape = [*, witdh, grid ** 2]
    x = x.permute(0, 2, 1) # shape = [*, grid ** 2, width]

    # expanding the CLS embedding?
    cls_embs = self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device)
    x = torch.cat([cls_embs, x], dim=1) #shape = [*, grid ** 2 + 1, width]

    x = x + self.positional_embedding.to(x.dtype)
    return x


In [ ]:
# ex
patchpos_embedding = PatchPostionalEmbeddingCLSToken()
x = torch.randn(1, 3, 224, 224)
out = patchpos_embedding(x)
print(out.shape)

In [ ]:
# class
class ViTTransformerEncoder(nn.Module):
  def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1) -> None:
    super().__init__()
    self.norm_1 = nn.LayerNorm(normalized_shape=embed_dim, eps=1e-6)
    self.attn = nn.MultiheadAttention(
        embed_dim=embed_dim,
        num_heads=num_heads,
        batch_first=True
    )
    self.norm_2 = nn.LayerNorm(normalized_shape=embed_dim, eps=1e-6)
    self.ffn = nn.Sequential(
        nn.Linear(in_features=embed_dim, out_features=ff_dim, bias=True),
        nn.GELU(),
        nn.Linear(in_features=ff_dim, out_features=embed_dim, bias=True)
    )
    self.dropout_1 = nn.Dropout(p=dropout)
    self.dropout_2 = nn.Dropout(p=dropout)

  def forward(self, x):
    # LayerNorm before Multi-Head Attention
    norm_x1 = self.norm_1(x)
    attn_output, _ = self.attn(norm_x1, norm_x1, norm_x1)
    attn_output = self.dropout_1(attn_output)
    x = x + attn_output # Residual connection

    # LayerNorm before Feed-Forward Network
    norm_x2 = self.norm_2(x)
    ffn_output = self.ffn(norm_x2)
    ffn_output = self.dropout_2(ffn_output)
    x = x + ffn_output # Residual connection

    return x

In [ ]:
class VisionTransformerCLS(nn.Module):
  def __init__(self,
               image_size, embed_dim, num_heads, ff_dim,
               dropout=0.1, num_classes=10, patch_size=16) -> None:
    super().__init__()
    self.embed_layer = PatchPostionalEmbeddingCLSToken

## Visualization